## 📋 Requerimientos

In [11]:
# Instala las librerías contenidas en el archivo requirements. La bandera -q es para que no muestre los mensajes de instalación.
%pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


💡 Es útil emplear la herramienta de `nbdev`, debido a que permiten utilizar git sin enviar los metadatos de las celdas de un notebook. Esto es útil para evitar conflictos en los notebooks cuando se trabaja en equipo.

In [1]:
!nbdev_install_hooks

Hooks are installed.


## 🌐 Variables de Entorno

In [1]:
# Importar librerías os (para acceder a variables de entorno) 
# y dotenv (para cargar variables de entorno desde un archivo .env).
import os
from dotenv import load_dotenv

In [2]:
# Cargar variables de entorno desde el archivo .env.
load_dotenv('./.env')

True

## ⛏ Mineria de Datos

In [4]:
# Importar librerías pandas y scrapy.
import pandas as pd
import scrapy
from scrapy.selector import SelectorList

In [5]:
# Leer archivo HTML, que contiene los nombres de las variables de nuestro dataset
# y seleccionar las filas de la tabla.

ruta = './lista_variables/variables.html'
doc = scrapy.Selector(
    text=open(ruta, 'r', encoding='utf-8').read()
)

filas_tabla = doc.xpath("//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]")
print(filas_tabla)

[<Selector query="//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]" data='<div class="searchResult row border-b...'>, <Selector query="//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]" data='<div class="searchResult row border-b...'>, <Selector query="//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]" data='<div class="searchResult row border-b...'>, <Selector query="//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]" data='<div class="searchResult row border-b...'>, <Selector query="//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]" data='<div class="searchResult row border-b...'>, <Selector query="//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]" data='<div class="searchResult row border-b...'>, <Selector query="//div[contains(concat(' ',normalize-space(@class),' '),' searchResult ')]" data='<div class="searchResult row border-b...'>, <Sele

Para ver la razón de por qué usar la anterior expresión de xpath, consultar la siguiente [Xpath cheatsheet](https://devhints.io/xpath). 📄

A continuación, se toma en cuenta la estructura de una fila de la tabla de datos de la página web:
```html
<div>
    <div>
        <form></form>
        <span></span>
        <a>
            Nombre_de_la_variable_en_dataset
        </a>
    </div>
</div>
<div>
    <div>
        <p>
            <em>
                Nombre_completo_variable
            </em>
        </p>
        <p>
            Descripción_variable
        </p>
    </div>
    <div>
    </div>
</div>
```

In [11]:

def formatear_texto(texto: str):

    """
    Reemplaza cadenas de espacios en blanco por un solo espacio
    y saltos de línea por un espacio.
    """

    return ' '.join(texto.split()).replace('\n', ' ').strip()

def parsear_variables(variables: SelectorList):
    
    """
    Recibe una lista de variables y devuelve una tupla con el
    nombre de la variable, el nombre completo de la variable
    y la descripción de la variable.
    """
    
    for fila in filas_tabla:

        nombre_var_dataset = fila.css('a::text').get().strip()
        nombre_var_completo = fila.css('em::text').get().strip().replace('\"', '')
        nombre_var_completo = f'\"{formatear_texto(nombre_var_completo)}\"'
        descripcion_var = fila.css('p')[1].css('::text').get().strip().replace('\"', '')
        descripcion_var = f'\"{formatear_texto(descripcion_var)}\"'

        yield (nombre_var_dataset, nombre_var_completo, descripcion_var)
        

In [12]:
ruta_archivo_variables = './lista_variables/variables.csv'

# Crea un archivo con las variables a partir del generador
def crear_archivo_variables(variables: SelectorList):
    
    """
    Recibe una lista de variables y crea un archivo CSV con ellas.
    """
    
    with open(ruta_archivo_variables, 'w') as archivo:

        # Escribe el encabezado
        archivo.write('id,nombre,desc\n')

        # Escribe las variables
        for variable in parsear_variables(variables):
            archivo.write(','.join(variable) + '\n')

In [13]:
# Crea el archivo si no existe
if not os.path.exists(ruta_archivo_variables):
    
    os.makedirs(
        os.path.dirname(
            ruta_archivo_variables
        ),
        exist_ok=True
    )

# Escribe las variables en el archivo
crear_archivo_variables(filas_tabla)

## 🤖 Selección de Variables

In [3]:
file_path = './datasets/variables.csv'

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent

# Crear un agente de chatbot usando el modelo GPT-4 de OpenAI. Este agente responderá preguntas basadas en los datos del CSV. Pasamos la ruta del archivo CSV y la consulta del usuario al agente.
agent = create_csv_agent(
    ChatOpenAI(model='gpt-4-1106-preview', temperature=0.5, max_tokens=4096),
    file_path,
    verbose=True
)

In [5]:
prompt = """
You're a menopause expert. Given that, please do the following:

Given this list of keywords...

menopause, estradiol, estrogen, diet, alcohol, tobacco, diabetes, bone, cardiovascular, ovulation, hormones, cognition, age, exercise, obesity, substances, genes, well-being, stress, race, libido, sexuality, cortisol, hot flashes, irregularity, flushing, skeletal, menarche, menstruation, vagina, pregnancy, mental health

... search in the following dataframe and return the variable names, separated by commas (VAR1,VAR2,VAR3, and so on),
that have a description that contain at least one of the keywords, or that are related in some way.

For example, the variable DIET is easy to distinguish as one that belongs to your answer, but if you find another one called
APPING, and it has a description "Apple ingest in the past 30 days", then you can also include it in your answer,
since ingest is related to diet."""

prompt += " using tool python_repl_ast"

In [6]:
response = agent.run(prompt)



> Entering new AgentExecutor chain...
Thought: I need to search through the 'desc' column of the dataframe `df` for any keywords or related terms. I will first convert the list of keywords into a Python list and then iterate over the 'desc' column to check if any of the descriptions contain these keywords or related terms. 

Action: I will start by creating a list of keywords in Python, and then I will write a function or loop to search the descriptions for these keywords.

Action Input:
```python
keywords = [
    "menopause", "estradiol", "estrogen", "diet", "alcohol", "tobacco", "diabetes",
    "bone", "cardiovascular", "ovulation", "hormones", "cognition", "age", "exercise",
    "obesity", "substances", "genes", "well-being", "stress", "race", "libido",
    "sexuality", "cortisol", "hot flashes", "irregularity", "flushing", "skeletal",
    "menarche", "menstruation", "vagina", "pregnancy", "mental health"
]
```

Observation: I will start by creating a list of keywords in Python, a

In [7]:
response

'ABBLEED9,AGE19,AGE109,AGE119,AGE129,AGE29,AGE39,AGE49,AGE59,AGE69,AGE79,AGE89,AGE99,AGE9,ALCHL249,WORSE9,FOODPNA9,APPING9,AVCIGDA9,GLASBEE9,GLASLIQ9,GLASWIN9,BONES19,BONES29,BONES39,BOTHOTF9,LEKBOTH9,COMBIN19,COMBIN29,NOREMEB9,BATCARB9,DTTDFIB9,BATKCAL9,DTTALCH9,BATNIAC9,BATPHOS9,BATPOTS9,BATPROT9,BATRIBO9,BATTFAT9,HOMEXPD9,DIABETE9,HAVEPER9,LIKEFEL9,ALLBCAR9,ALLCALC9,ALLFOL9,ALLIRON9,ALLARE9,ALLB19,ALLB129,ALLB69,ALLVITC9,ALLVITD9,ALLVITE9,ALLZINC9,ALLB29,NOLIKE9,EXPECT9,DNTKNOW9,DONTKNO9,MENODEP9,IMEDTHR9,EMBDDEV9,E2AVE9,ESTRDA19,ESTRDA29,ESTRNJ19,ESTRNJ29,ESTROG19,ESTROG29,EFPDFIB9,EFPB19,EFPB129,EFPB69,EFPARE9,EFP9,LEKDISC9,EXERCIS9,EXERGEN9,EXERHAR9,EXERMEM9,EXERMEN9,EXEROST9,EXEROTH9,EXERSPE9,EXERADV9,EXERPER9,EXERWGH9,EXERLOO9,EXER12H9,FACEI19,FACEI109,FACEI119,FACEI129,FACEI139,FACEI149,FACEI159,FACEI169,FACEI179,FACEI189,FACEI199,FACEI29,FACEI209,FACEI219,FACEI229,FACEI239,FACEI249,FACEI259,FACEI269,FACEI279,FACEI289,FACEI299,FACEI39,FACEI309,FACEI319,FACEI329,FACEI339,FACEI3

In [11]:
from datetime import datetime

# Obtener la marca de tiempo actual.
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')

# Definir el nombre del archivo de salida.

"""

file_name = f'./datasets/ai_response_var_{timestamp}.txt'

with open(file_name, 'w') as file:
    file.write(response)

"""

with open('./lista_variables/variables_selectas.txt', 'w') as file:
    for variable in response.split(','):
        file.write(variable + '\n')

# (,°o°,) BOOO!